In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
while not os.getcwd().endswith('Laboratorio_SiPM'):
    os.chdir('..')

import sys
sys.path.append('Config')
from lablib import *

# Ladder

In [ ]:
def invert (X):
    for i in range(len(X)):
        X[i] = - X[i]
    return X

In [ ]:
def graph(X1, Y1, X2, Y2, X3, Y3, X4, Y4, X5, Y5, X6, Y6):
    fig, ax = plt.subplots(nrows= 1, ncols=1)

    plt.plot(invert(X1), Y1, label='53.5mV', linewidth=0.8)
    ax.scatter(X1, Y1)

    plt.plot(invert(X2), Y2, label='54mV', linewidth=0.8)
    ax.scatter(X2, Y2)

    plt.plot(invert(X3), Y3, label='54.5mV',linewidth=0.8)
    ax.scatter(X3, Y3)
    
    plt.plot(invert(X5), Y5, label='55mV',linewidth=0.8)
    ax.scatter(X5, Y5)
    
    plt.plot(invert(X6), Y6, label='55.5mV',linewidth=0.8)
    ax.scatter(X6, Y6)

    ax.set_yscale ("log")
    ax.set_xlabel("Threshold[mV]")
    ax.set_ylabel("Log(Frequency[kHz])")
    ax.legend()
    plt.grid()
    plt.savefig("Staircase_Finale.png")
    plt.show()
   

In [ ]:
def cut_0(X, Y):
    X1 = []
    Y1 = []
    for i in range(len(X)):
        if X[i] != 0:
            X1.append(X[i])
            Y1.append(Y[i])
    return Y1, X1

In [ ]:
Z = pd.read_csv(r"Analisi/Parte 1/Analisi Dark_Ladder/staircase_plot_bello.CSV")
Y = Z["Th [mV] V_B 53_5V "]
K1 = cut_0(Z["F [kHz] V_B 53_5V"], Y)
K2 = cut_0(Z["F [kHz] V_B 54V"], Y)
K3 = cut_0(Z["F [kHz] V_B 54_5V"], Y)
K4 = cut_0(Z["F [kHz] V_B 55V"], Y)
K5 = cut_0(Z["F [kHz] V_B 55_5V"], Y)
K6 = cut_0(Z["F [kHz] V_B 56V"], Y)

print(K1[1])

graph(K1[0], K1[1], K2[0], K2[1], K3[0], K3[1], K4[0], K4[1], K5[0], K5[1], K6[0], K6[1])

# Crosstalk

In [ ]:
# ct_53_5 difficilmente distinguibile

ct_54 = K2[1][10] / K2[1][5]
ct_54_5 = K3[1][12] / K3[1][6]
ct_55 = K5[1][14] / K5[1][7]
ct_55_5 = K6[1][17] / K6[1][10]

x = np.array([54, 54.5, 55, 55.5])
y = np.array([ct_54, ct_54_5, ct_55, ct_55_5])
yer = [.001 for _ in range(len(x))]

vbr = 52

In [ ]:
def exp(x, a, b):
    return a * (1 - np.exp(x / b))

pars, vals, ers, *other = LS_fit(x, y, yer, exp, a=1, b=1)

for i in range(len(pars)):
    print(f"Parameter {pars[i]}: {vals[i]:.3g} ± {ers[i]:.3g}")

In [ ]:
t = np.linspace(54, 55.5, 100)
plt.errorbar(x-vbr, y, yerr=yer, fmt='o', label='Data', color='blue')
plt.plot(t-vbr, exp(np.array(t), *vals), label='Fit', color='red')
plt.xlabel('Overvoltage [V]')
plt.ylabel('Cross-talk Probability')
plt.grid()
plt.legend()
plt.show()